<img src="http://meusite.mackenzie.br/rogerio/mackenzie_logo/UPM.2_horizontal_vermelho.jpg"  width=300, align="right">
<br>
<br>
<br>
<br>
<br>

# **Template para o Colab do Projeto Semestral**
---

Atenção, podem ser que nem todas as tarefas sejam executadas no Colab (a aplicação por exemplo, pode estar hospedada no streamlit cloud). Mas a maior parte pode estar aqui ou ao menos indicada e comentada.

In [ ]:
#@title **Identificação do Grupo**

#@markdown Integrantes do Grupo, nome completo em orgem alfabética (*informe \<RA\>,\<nome\>*)
Aluno1 = '10409941, Carlos Eduardo Rosendo Basseto' #@param {type:"string"}
Aluno2 = '10408953, Matheus Santiago de Brito ' #@param {type:"string"}




# Detecção de discurso ofensivo em textos em português

Este notebook implementa um fluxo completo de **Machine Learning** para detectar discurso ofensivo em comentários de texto, usando o dataset **OLID-BR**.

Etapas principais:
1. Carregamento e preparação do dataset `dougtrajano/olid-br` (Hugging Face).
2. Criação de um arquivo CSV com as colunas `texto` e `label`.
3. Separação em treino e teste.
4. Balanceamento do conjunto de treino (undersampling da classe ofensiva).
5. Treinamento e avaliação de três modelos:
   - KNN
   - Árvore de Decisão
   - MLPClassifier (rede neural)
6. Salvamento do modelo final para uso em uma aplicação Streamlit.


## 1. Carregando o dataset OLID-BR

Nesta célula, vamos:
- Baixar o dataset `dougtrajano/olid-br` do Hugging Face;
- Unir os conjuntos `train` e `test` em um único `DataFrame` do pandas;
- Visualizar as colunas disponíveis.


In [ ]:
from datasets import load_dataset
import pandas as pd
import os

# 1. Carregar dataset OLID-BR
dataset_id = "dougtrajano/olid-br"
ds = load_dataset(dataset_id)

print(ds)

In [ ]:
df_train = ds["train"].to_pandas()
df_test  = ds["test"].to_pandas()

df = pd.concat([df_train, df_test], ignore_index=True)

print("Colunas disponíveis:")
print(df.columns)

df.head()

## 2. Selecionando colunas relevantes e criando a coluna `label`

Aqui vamos:
- Manter apenas as colunas `text` (comentário) e `is_offensive` (rótulo original);
- Renomear `text` para `texto`;
- Transformar `is_offensive` em uma coluna numérica `label`, onde:
  - `1` = ofensivo (`OFF`)
  - `0` = não ofensivo (`NOT`);
- Ver a distribuição das classes.


In [ ]:
# Vamos usar apenas o texto e a info se é ofensivo ou não
df = df[["text", "is_offensive"]].copy()

# Renomear coluna de texto para o padrão do projeto
df = df.rename(columns={"text": "texto"})

# Mapear OFF/NOT para 1/0
def map_offensive(x):
    return 1 if x == "OFF" else 0

df["label"] = df["is_offensive"].apply(map_offensive)

print("Distribuição das classes (label):")
print(df["label"].value_counts())
print(df["label"].value_counts(normalize=True))

df.head()

## 3. Salvando o dataset preparado em CSV

Agora salvamos o DataFrame em um arquivo CSV que será usado nas próximas etapas do projeto.


In [ ]:
import os

os.makedirs("../data", exist_ok=True)

caminho_csv = "../data/dados_hatespeech.csv"
df.to_csv(caminho_csv, index=False, encoding="utf-8")

print("CSV salvo em:", caminho_csv)

## 4. Importando bibliotecas de modelagem e carregando o CSV

Nesta célula:
- Importamos as bibliotecas necessárias para treinamento e avaliação dos modelos;
- Carregamos o arquivo CSV gerado na etapa anterior.


In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix
)

# Modelos que vamos testar
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

import joblib
import os

# Carregar o CSV que você salvou
caminho_csv = "/data/dados_hatespeech.csv"
df = pd.read_csv(caminho_csv)

print("Primeiras linhas:")
df.head()

## 5. Separando conjunto de treino e teste

Aqui separamos:
- `X` = textos (`texto`)
- `y` = rótulos (`label`)

Usamos `train_test_split` para criar:
- 80% para treino
- 20% para teste

mantendo a proporção original das classes (parâmetro `stratify`).


In [ ]:
# Separar features e rótulos
X = df["texto"].astype(str)
y = df["label"].astype(int)

# Separar em treino/teste (80% / 20%)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y   # mantém proporção das classes
)

print("Tamanho do treino:", len(X_train))
print("Tamanho do teste :", len(X_test))

## 6. Balanceando o conjunto de treino (undersampling)

O dataset original tem muito mais exemplos ofensivos do que não ofensivos.
Nesta célula:
- Criamos um DataFrame com `X_train` e `y_train`;
- Contamos a distribuição original;
- Fazemos **undersampling** da classe ofensiva para ter o mesmo número de exemplos que a classe não ofensiva;
- Embaralhamos os dados e atualizamos `X_train` e `y_train` com a versão balanceada.


In [ ]:
# Juntar X_train e y_train em um DataFrame
train_df = pd.DataFrame({
    "texto": X_train,
    "label": y_train
})

print("Distribuição original no treino:")
print(train_df["label"].value_counts())

# Separar por classe
train_normal = train_df[train_df["label"] == 0]
train_ofensivo = train_df[train_df["label"] == 1]

# Fazer undersampling da classe ofensiva para ter o mesmo número de exemplos que a classe normal
train_ofensivo_down = train_ofensivo.sample(
    n=len(train_normal),
    random_state=42
)

# Juntar e embaralhar
train_bal = pd.concat([train_normal, train_ofensivo_down]).sample(frac=1, random_state=42)

print("\nDistribuição balanceada no treino:")
print(train_bal["label"].value_counts())

# Atualizar X_train e y_train para usar o conjunto balanceado
X_train = train_bal["texto"]
y_train = train_bal["label"]

## 7. Função para treinar e avaliar modelos

Para evitar repetição de código, criamos a função `treinar_e_avaliar`, que:
- Monta um `Pipeline` com `TfidfVectorizer` + modelo;
- Treina o modelo no conjunto de treino;
- Avalia no conjunto de teste;
- Imprime acurácia e relatório de classificação;
- Mostra a matriz de confusão;
- Retorna o pipeline treinado e um dicionário com métricas (acurácia e F1 da classe ofensiva).


In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
import seaborn as sns
import matplotlib.pyplot as plt

def treinar_e_avaliar(nome_modelo, modelo_base):
    """
    nome_modelo: Apenas para exibição
    modelo_base: O estimador (KNN, Árvore, MLP etc.)
    """
    print(f"\n================ {nome_modelo} ================\n")

    # Pipeline: TF-IDF + modelo
    pipeline = Pipeline([
        ("tfidf", TfidfVectorizer(
            lowercase=True,
            ngram_range=(1, 2),      # unigram + bigram
            max_features=20000       # limita tamanho do vocabulário
        )),
        ("clf", modelo_base)
    ])

    # Treinar
    pipeline.fit(X_train, y_train)

    # Predizer
    y_pred = pipeline.predict(X_test)

    # Acurácia
    acc = accuracy_score(y_test, y_pred)
    print(f"Acurácia: {acc:.4f}\n")

    # Relatório completo
    print("Relatório de classificação:")
    print(classification_report(y_test, y_pred, digits=4))

    # Matriz de confusão
    cm = confusion_matrix(y_test, y_pred)

    plt.figure(figsize=(4, 4))
    sns.heatmap(
        cm,
        annot=True,
        fmt="d",
        cmap="Blues",
        xticklabels=["Normal", "Ofensivo"],
        yticklabels=["Normal", "Ofensivo"]
    )
    plt.xlabel("Predito")
    plt.ylabel("Real")
    plt.title(f"Matriz de Confusão - {nome_modelo}")
    plt.tight_layout()
    plt.show()

    # F1-score da classe OFENSIVO (1)
    report = classification_report(y_test, y_pred, output_dict=True)
    f1_classe_1 = report["1"]["f1-score"]

    return pipeline, {
        "nome": nome_modelo,
        "acuracia": acc,
        "f1_ofensivo": f1_classe_1
    }

## 8. Treinando e avaliando o modelo KNN

Primeiro modelo: **KNN (k=5)**.

Aqui chamamos a função `treinar_e_avaliar` passando o KNN como estimador.


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Modelo KNN com k = 5 vizinhos
knn = KNeighborsClassifier(n_neighbors=5)

modelo_knn, resultados_knn = treinar_e_avaliar("KNN (k=5)", knn)

print("\nResultados KNN:", resultados_knn)

## 9. Treinando e avaliando o modelo Árvore de Decisão

Agora treinamos uma **Árvore de Decisão** com profundidade máxima limitada para tentar evitar overfitting.


In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Árvore com profundidade máxima para evitar overfitting
tree = DecisionTreeClassifier(
    max_depth=20,
    random_state=42
)

modelo_tree, resultados_tree = treinar_e_avaliar("Árvore de Decisão (max_depth=20)", tree)

print("\nResultados Árvore:", resultados_tree)

## 10. Treinando e avaliando o modelo MLPClassifier (rede neural)

Por fim, treinamos uma rede neural simples (`MLPClassifier`) com duas camadas ocultas.
Este foi o modelo que apresentou melhor desempenho.


In [ ]:
from sklearn.neural_network import MLPClassifier

# Rede MLP com 2 camadas ocultas
mlp = MLPClassifier(
    hidden_layer_sizes=(64, 32),
    max_iter=10,  # poucas iterações por causa do custo
    random_state=42,
    verbose=True
)

modelo_mlp, resultados_mlp = treinar_e_avaliar("MLPClassifier (64,32)", mlp)

print("\nResultados MLP:", resultados_mlp)

## 11. Salvando o modelo final para uso no Streamlit

Aqui salvamos o `pipeline` treinado (TF-IDF + MLP) em um arquivo `.pkl`,
que será carregado pela aplicação Streamlit.


In [ ]:
import joblib
import os

# Criar pasta models (se ainda não existir)
os.makedirs("./models", exist_ok=True)

# Caminho do arquivo do modelo
caminho_modelo = "./models/modelo_hatespeech_mlp.pkl"

# Salvar o pipeline inteiro (TF-IDF + MLP)
joblib.dump(modelo_mlp, caminho_modelo)

print("Modelo salvo em:", caminho_modelo)


# **Referências**

1. Dataset OLID-BR
Trajano, D. (2022). OLID-BR: Offensive Language Identification Dataset in Brazilian Portuguese. Disponível em:
https://huggingface.co/datasets/dougtrajano/olid-br

2. TF-IDF — Vetorização de Texto
Ramos, J. (2003). Using TF-IDF to Determine Word Relevance in Document Queries. Proceedings of the First Instructional Conference on Machine Learning.

3. Scikit-Learn — Modelos de Machine Learning
Pedregosa, F. et al. (2011). Scikit-learn: Machine Learning in Python. Journal of Machine Learning Research.
https://scikit-learn.org/

4. Streamlit — Interface da Aplicação
Streamlit Documentation.
https://docs.streamlit.io/

5. Catálogo de dados de discurso de ódio
Vidgen, B., & Derczynski, L. (2020). Datasets for abusive language detection. PLoS ONE.
https://hatespeechdata.com